# Description

It standardizes the features (latent variables) of an input file.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from IPython.display import display

import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

import conf

# Settings

In [3]:
INPUT_FILEPATH = Path(
    conf.RESULTS['PROJECTIONS_DIR'],
    'projection-smultixcan-mashr-zscores.pkl'
).resolve()
display(INPUT_FILEPATH)

input_filepath_stem = INPUT_FILEPATH.stem
display(input_filepath_stem)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/projections/projection-smultixcan-mashr-zscores.pkl')

'projection-smultixcan-mashr-zscores'

In [4]:
# output dir for this notebook
RESULTS_DIR = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    'z_score_std'
).resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/z_score_std')

# Load input file

In [5]:
data = pd.read_pickle(INPUT_FILEPATH)

In [6]:
data.shape

(987, 4091)

In [7]:
data.head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
LV1,-0.026255,0.006045,0.007276,-0.029740,-0.001949,0.009288,0.006488,0.003354,-0.021751,-0.039159,...,0.036648,0.094062,0.047505,-0.035089,-0.036600,0.185595,-0.031841,-0.011883,0.033163,0.061374
LV2,-0.006253,-0.023996,0.037932,-0.000018,-0.016128,-0.033736,-0.017215,-0.004558,0.079234,-0.040277,...,0.085747,-0.064254,0.022124,-0.008441,-0.049594,0.074838,-0.057865,-0.007585,0.003580,-0.025216
LV3,-0.002729,0.051637,-0.013182,-0.001663,0.035221,-0.016019,0.000484,-0.046965,-0.000498,-0.041794,...,0.068618,-0.034395,0.145881,0.031539,-0.031740,0.015501,-0.028148,-0.048226,0.030093,-0.026938
LV4,0.042319,-0.023236,0.063016,-0.002628,-0.046487,0.018004,-0.029841,-0.023949,-0.026485,0.022759,...,-0.070986,-0.028960,0.015369,0.051671,0.014078,-0.147863,0.034195,0.068829,-0.023213,-0.089619
LV5,-0.025975,0.002167,-0.022646,0.004559,0.004649,0.028440,-0.024379,-0.028536,0.019263,-0.026291,...,0.008842,-0.000932,0.004195,0.029759,-0.043760,0.057031,-0.002836,-0.046215,0.099309,0.075663


# Data preprocessing for clustering

In [8]:
data_stats = data.T.iloc[:, :10].describe()
display(data_stats)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10
count,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03
mean,-1.791120e-18,-3.785776e-18,3.039476e-18,-3.365135e-18,4.722044e-18,-4.342109e-19,4.233556e-18,-9.226982e-19,-3.799345e-19,-7.598691e-19
std,3.326184e-02,2.835837e-02,3.706188e-02,3.046931e-02,3.007658e-02,2.642815e-02,3.693083e-02,3.091832e-02,3.174964e-02,2.954993e-02
min,-1.800493e-01,-9.502399e-02,-8.809295e-02,-2.374265e-01,-8.089705e-02,-2.076498e-01,-8.773850e-02,-8.628219e-02,-8.309656e-02,-1.209542e-01
25%,-2.013433e-02,-1.830758e-02,-2.164229e-02,-1.893100e-02,-2.037183e-02,-1.703704e-02,-2.199054e-02,-1.916786e-02,-1.824650e-02,-1.945359e-02
50%,-2.566123e-03,-1.081795e-03,-4.677365e-03,-1.997775e-04,-2.374624e-03,-2.582970e-04,-4.157257e-03,-3.233265e-03,-2.972257e-03,-1.338313e-03
75%,1.693591e-02,1.698605e-02,1.363002e-02,1.819961e-02,1.563033e-02,1.674753e-02,1.502715e-02,1.409039e-02,1.354585e-02,1.774809e-02
max,3.304012e-01,2.484342e-01,3.385425e-01,1.884026e-01,3.158933e-01,1.094037e-01,3.772630e-01,2.799561e-01,3.681587e-01,2.264011e-01


## Standardize

In [9]:
data_t = data.T

scaled_data = pd.DataFrame(
    data=scale(data_t),
    index=data_t.index.copy(),
    columns=data_t.columns.copy()
)

In [10]:
scaled_data.shape

(4091, 987)

In [11]:
scaled_data.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
20096_1-Size_of_red_wine_glass_drunk_small_125ml,-0.789441,-0.220511,-0.073647,1.389075,-0.863722,0.837183,0.115059,-0.489714,-0.060237,-0.739970,...,-0.828369,-0.766238,-0.795901,-0.060845,-0.650635,0.198545,-1.046268,-0.907707,0.119110,0.090268
2345-Ever_had_bowel_cancer_screening,0.181777,-0.846275,1.393447,-0.762712,0.072048,-0.374755,-0.977721,0.283071,0.149015,1.216908,...,1.036607,0.533858,0.900654,1.301938,0.098157,0.247145,1.076262,-1.307636,1.550653,0.036361
N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,0.218786,1.337743,-0.355718,2.068429,-0.753050,0.519698,0.603722,-0.656698,-0.183306,-0.889235,...,0.070416,-0.226306,-0.140882,0.557099,-1.195313,1.047936,-0.235122,0.735247,0.199132,-0.103762
100011_raw-Iron,-0.894239,-0.000629,-0.044876,-0.086266,0.151583,0.996284,0.009040,1.871274,-0.357051,1.240719,...,0.594991,0.923486,-0.723010,0.527064,0.449430,-0.741877,0.345336,1.168521,-1.740056,-0.916040
5221-Index_of_best_refractometry_result_right,-0.058612,-0.568793,0.950456,-1.525888,0.154586,-1.224318,0.066917,0.211487,0.044214,1.328202,...,1.298788,-0.204407,-0.427908,0.429530,-0.359803,-0.872641,-0.516132,-0.881260,0.404037,0.480434


In [12]:
scaled_data_stats = scaled_data.iloc[:,:10].describe()
display(scaled_data_stats)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10
count,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4091.000000,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03
mean,3.473687e-18,1.867107e-17,1.389475e-17,-2.257897e-17,0.000000,-8.684218e-18,-5.210531e-18,-1.259212e-17,-1.736844e-18,-1.302633e-17
std,1.000122e+00,1.000122e+00,1.000122e+00,1.000122e+00,1.000122,1.000122e+00,1.000122e+00,1.000122e+00,1.000122e+00,1.000122e+00
min,-5.413752e+00,-3.351237e+00,-2.377206e+00,-7.793270e+00,-2.690031,-7.858105e+00,-2.376042e+00,-2.790990e+00,-2.617564e+00,-4.093716e+00
25%,-6.054023e-01,-6.456582e-01,-5.840214e-01,-6.213897e-01,-0.677415,-6.447337e-01,-5.955249e-01,-6.200274e-01,-5.747697e-01,-6.584099e-01
50%,-7.715859e-02,-3.815195e-02,-1.262196e-01,-6.557481e-03,-0.078962,-9.774751e-03,-1.125825e-01,-1.045872e-01,-9.362690e-02,-4.529544e-02
75%,5.092317e-01,5.990515e-01,3.678088e-01,5.973826e-01,0.519748,6.337779e-01,4.069496e-01,4.557852e-01,4.266980e-01,6.006871e-01
max,9.934556e+00,8.761596e+00,9.135635e+00,6.184114e+00,10.504248,4.140171e+00,1.021664e+01,9.055807e+00,1.159710e+01,7.662582e+00


In [13]:
assert np.all([np.isclose(scaled_data_stats.loc['mean', c], 0.0) for c in scaled_data_stats.columns])

In [14]:
assert np.all([np.isclose(scaled_data_stats.loc['std', c], 1.0, atol=1e-03) for c in scaled_data_stats.columns])

# Save

In [15]:
output_file = Path(
    RESULTS_DIR,
    f'z_score_standardized-{input_filepath_stem}.pkl',
).resolve()

display(output_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/z_score_std/z_score_standardized-projection-smultixcan-mashr-zscores.pkl')

In [16]:
scaled_data.to_pickle(output_file)